## Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import gym
from scipy.special import softmax
import torch.nn.functional as F
import random
from collections import deque, namedtuple
import torch.optim as optim
import matplotlib.pyplot as plt
import itertools
import pandas as pd

## Environments

### Cartpole

In [ ]:
env_cartpole = gym.make("CartPole-v1")

### Acrobot

In [ ]:
env_acrobot = gym.make("Acrobot-v1")

## Helper Functions (run after Dueling DQN classes, before environment grid search and inference)

Grid Search function that takes in a set of parameters to search over and finds the best set by minimizing average regret over 5 seeds for each set of parameters. **These helper functions should be run after the DDQN classes have been run**

In [ ]:
results_df = pd.DataFrame({"buffer_size" : [0], "batch_size": [0], "gamma": [0], "lr": [0], "update_every": [0], "regret":[0]})

In [ ]:
def grid_search(env, env_vars, general_vars, policy, update_type):

  var_lists = env_vars.values()
  gen_var_lists = general_vars[policy].values()

  env_grid = list(itertools.product(*var_lists))
  gen_grid = list(itertools.product(*gen_var_lists))
  grid = [[*x, *y] for x in env_grid for y in gen_grid]
  grid_search_results = []

  best_regret = 1e9

  state_shape = env.observation_space.shape[0]
  action_shape = env.action_space.n

  i=0
  seeds = [0, 1, 2, 3, 4]

  if(policy=="Epsilon Greedy"):

    for combination in grid:
      i+=1
      buffer_size, batch_size, gamma, lr, update_every, start, end, decay = combination
      print(f"combination: {combination} , number: {i}")
      regret = 0

      for seed in seeds:
        agent = Agent(state_shape, action_shape, seed, update_type, policy, buffer_size,
                              lr, gamma, update_every, batch_size)
        _, scores, regrets = ddqn(env, 500, 500, start, end, decay, 1, 1, 1, agent)
        regret += np.mean(regrets)

      regret/=5

      if regret < best_regret:
        best_regret = regret
        grid_search_results = [buffer_size, batch_size, gamma, lr, update_every, start, end, decay]

      results_df.loc[len(results_df)] = [buffer_size, batch_size, gamma, lr, update_every, regret]

      print(f"Regret - {regret}")

  elif(policy=="Softmax"):

    for combination in grid:
      i+=1
      buffer_size, batch_size, gamma, lr, update_every, start, end, decay = combination
      print(f"combination: {combination} , number: {i}")
      regret = 0

      for seed in seeds:
        agent = Agent(state_shape, action_shape, seed, update_type, policy, buffer_size,
                              lr, gamma, update_every, batch_size)
        _, scores, regrets = ddqn(env, 500, 500, 1, 1, 1, start, end, decay, agent)
        regret += np.mean(regrets)

      regret/=5

      if regret < best_regret:
        best_regret = regret
        grid_search_results = [buffer_size, batch_size, gamma, lr, update_every, start, end, decay]

      results_df.loc[len(results_df)] = [buffer_size, batch_size, gamma, lr, update_every, regret]

      print(f"Regret - {regret}")

  return grid_search_results, best_regret

Next we have a plot function that plots the regrets, scores and running scores for each type of update rule over 5 seeds. This function also computes the mean and std over the seeds before plotting.

In [ ]:
def plot(regrets_list_type1, scores_list_type1, regrets_list_type2, scores_list_type2, running_scores_1, running_scores_2, reward_threshold): #extended_rewards,

  max_eps = max(len(lst) for lst in regrets_list_type1)
  extended_regrets_type1 = [lst + [0] * (max_eps - len(lst)) for lst in regrets_list_type1]
  extended_scores_type1 = [lst + [0] * (max_eps - len(lst)) for lst in scores_list_type1]
  extended_regrets_type2 = [lst + [0] * (max_eps - len(lst)) for lst in regrets_list_type2]
  extended_scores_type2 = [lst + [0] * (max_eps - len(lst)) for lst in scores_list_type2]
  r1 = np.empty((max_eps, 5))
  s1 = np.empty((max_eps, 5))
  r2 = np.empty((max_eps, 5))
  s2 = np.empty((max_eps, 5))
  rs1 = np.empty((max_eps, 5))
  rs2 = np.empty((max_eps, 5))

  for i in range(5):
    for j in range(max_eps):
      r1[j][i] = extended_regrets_type1[i][j]
      s1[j][i] = extended_scores_type1[i][j]
      r2[j][i] = extended_regrets_type2[i][j]
      s2[j][i] = extended_scores_type2[i][j]
      rs1[j][i] = running_scores_1[i][j]
      rs2[j][i] = running_scores_2[i][j]

  r1_mean_values = np.mean(r1, axis=1)
  r1_std_dev = np.std(r1, axis=1)
  s1_mean_values = np.mean(s1, axis=1)
  s1_std_dev = np.std(s1, axis=1)
  r2_mean_values = np.mean(r2, axis=1)
  r2_std_dev = np.std(r2, axis=1)
  s2_mean_values = np.mean(s2, axis=1)
  s2_std_dev = np.std(s2, axis=1)
  rs1_mean_values = np.mean(rs1, axis=1)
  rs1_std_dev = np.std(rs1, axis=1)
  rs2_mean_values = np.mean(rs2, axis=1)
  rs2_std_dev = np.std(rs2, axis=1)

  x = np.arange(max_eps)
  t = [reward_threshold]*max_eps

  plt.figure()
  plt.plot(x, r1_mean_values, color='blue', linestyle='-', label='Type 1 Update')
  plt.fill_between(x, r1_mean_values - r1_std_dev, r1_mean_values + r1_std_dev, color='lightblue')
  plt.plot(x, r2_mean_values, color='red', linestyle='-', label='Type 2 Update')
  plt.fill_between(x, r2_mean_values - r2_std_dev, r2_mean_values + r2_std_dev, color='mistyrose')
  plt.plot(x, t, color='green', linestyle='-', label='Threshold')
  plt.xlabel('Episode')
  plt.ylabel('Regret')
  plt.legend()
  plt.grid(True)
  plt.show()

  plt.figure()
  plt.plot(x, s1_mean_values, color='blue', linestyle='-', label='Type 1 Update')
  plt.fill_between(x, s1_mean_values - s1_std_dev, s1_mean_values + s1_std_dev, color='lightblue')
  plt.plot(x, s2_mean_values, color='red', linestyle='-', label='Type 2 Update')
  plt.fill_between(x, s2_mean_values - s2_std_dev, s2_mean_values + s2_std_dev, color='mistyrose')
  plt.plot(x, t, color='green', linestyle='-', label='Threshold')
  plt.xlabel('Episode')
  plt.ylabel('Reward')
  plt.legend()
  plt.grid(True)
  plt.show()

  plt.figure()
  plt.plot(x, rs1_mean_values, color='blue', linestyle='-', label='Type 1 Update')
  plt.fill_between(x, rs1_mean_values - rs1_std_dev, rs1_mean_values + rs1_std_dev, color='lightblue')
  plt.plot(x, rs2_mean_values, color='red', linestyle='-', label='Type 2 Update')
  plt.fill_between(x, rs2_mean_values - rs2_std_dev, rs2_mean_values + rs2_std_dev, color='mistyrose')
  plt.plot(x, t, color='green', linestyle='-', label='Threshold')
  plt.xlabel('Episode')
  plt.ylabel('Running Reward')
  plt.legend()
  plt.grid(True)
  plt.show()

Finally, we have an inference function that takes in the optimal set of hyperparameters and runs inference on each update type (taken in as an argument) in each environment (also an argument).

In [ ]:
def inference(env, vars, policy, update_type):

  state_shape = env.observation_space.shape[0]
  action_shape = env.action_space.n

  seeds = [0, 1, 2, 3, 4]
  regrets_list = []
  scores_list = []
  running_scores_list = []

  buffer_size, batch_size, gamma, lr, update_every, start, end, decay = vars

  if(policy=="Epsilon Greedy"):

    for seed in seeds:
      print(seed)
      agent = Agent(state_shape, action_shape, seed, update_type, policy, buffer_size,
                            lr, gamma, update_every, batch_size)
      _, scores, regrets, running_scores = ddqn(env, 500, 500, start, end, decay, 1, 1, 1, agent)
      regrets_list.append(regrets)
      scores_list.append(scores)
      running_scores_list.append(running_scores)

  elif(policy=="Softmax"):

    for seed in seeds:
      print(seed)
      agent = Agent(state_shape, action_shape, seed, update_type, policy, buffer_size,
                            lr, gamma, update_every, batch_size)
      _, scores, regrets, running_scores = ddqn(env, 500, 500, 1, 1, 1, start, end, decay, agent)
      regrets_list.append(regrets)
      scores_list.append(scores)
      running_scores_list.append(running_scores)

  return regrets_list, scores_list, running_scores_list

## Dueling DQN

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Define the general policy variables, start, end and decay values

In [ ]:
general_vars = {
    "Epsilon Greedy": {
        "eps_start" : [0.1],
        "eps_end" : {0.0001},
        "eps_decay" : {0.995}
    },
    "Softmax":{
        "temp_start" : [1],
        "temp_end" : {0.01},
        "temp_decay" : {0.995}
    }
}

The two update types are defined below as mentioned in the problem statement:

In [ ]:
def type1Update(advantages, values):
    return values + (advantages - advantages.mean())

def type2Update(advantages, values):
  return values + (advantages - torch.max(advantages))

The Q Network is defined below:

In [ ]:
class QNetwork(nn.Module):
  def __init__(self, input_dim, output_dim, seed):
    super(QNetwork, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim

    self.feauture_layer = nn.Sequential(
        nn.Linear(self.input_dim, 64),
        nn.ReLU()
    )

    self.value_stream = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 1)
    )

    self.advantage_stream = nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, self.output_dim)
    )

  def forward(self, state, update_type):
      features = self.feauture_layer(state)
      values = self.value_stream(features)
      advantages = self.advantage_stream(features)
      qvals = update_type(advantages, values)
      return qvals

Next, we have the replay buffer

In [ ]:
class ReplayBuffer():
  def __init__(self, action_size, buffer_size, batch_size, seed):
    self.action_size = action_size
    self.memory = deque(maxlen=buffer_size)
    self.batch_size = batch_size
    self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    self.seed = random.seed(seed)

  def add(self, state, action, reward, next_state, done):
    e = self.experience(state, action, reward, next_state, done)
    self.memory.append(e)

  def sample(self):
    experiences = random.sample(self.memory, k=self.batch_size)

    states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
    actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
    rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
    next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
    dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

    return (states, actions, rewards, next_states, dones)

  def __len__(self):
    return len(self.memory)

The agent class is defined as follows:

In [ ]:
class Agent():
  def __init__(self, state_size, action_size, seed, update_type, policy_type, buffer_size, lr, gamma, update_every, batch_size):
    self.state_size = state_size
    self.action_size = action_size
    self.seed = random.seed(seed)
    self.update_type = update_type
    self.policy_type = policy_type
    self.buffer_size = buffer_size
    self.batch_size = batch_size
    self.update_every = update_every
    self.lr = lr
    self.gamma = gamma
    self.seed=seed

    torch.manual_seed(seed)
    self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
    torch.manual_seed(seed)
    self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
    self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)

    self.memory = ReplayBuffer(action_size, self.buffer_size, self.batch_size, seed)

    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.add(state, action, reward, next_state, done)

    if len(self.memory) >= self.batch_size:
        experiences = self.memory.sample()
        self.learn(experiences, self.gamma)

    self.t_step = (self.t_step + 1) % self.update_every
    if self.t_step == 0:

        self.qnetwork_target.load_state_dict(self.qnetwork_local.state_dict())

  def act(self, state, eps=0., temp=1.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    self.qnetwork_local.eval()
    with torch.no_grad():
        action_values = self.qnetwork_local(state, self.update_type)
    self.qnetwork_local.train()

    if(self.policy_type == "Epsilon Greedy"):
      if random.random() > eps:
          return np.argmax(action_values.cpu().data.numpy())
      else:
          return random.choice(np.arange(self.action_size))

    else:
      action_values = action_values.cpu().data.numpy().reshape(action_values.shape[1]) / temp
      probabilities = softmax(action_values)
      return np.random.choice(len(probabilities), p=probabilities)

  def learn(self, experiences, gamma):
    states, actions, rewards, next_states, dones = experiences
    Q_targets_next = self.qnetwork_target(next_states, self.update_type).detach().max(1)[0].unsqueeze(1)
    Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

    Q_expected = self.qnetwork_local(states, self.update_type).gather(1, actions)

    loss = F.mse_loss(Q_expected, Q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

The dueling dqn algorithm is executed using the function below:

In [ ]:
def ddqn(env, n_episodes, max_t, eps_start, eps_end, eps_decay, temp_start, temp_end, temp_decay, agent):

    scores_window = deque(maxlen=100)
    eps = eps_start
    temp = temp_start

    scores, regrets = [], []
    running_avg_scores = []

    running_reward = 10 # change to -500 for acrobot

    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps, temp)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break

        scores_window.append(score)
        scores.append(score)
        regrets.append(500-score) # change to regrets.append(-score) for acrobot
        running_reward = 0.05 * score + (1 - 0.05) * running_reward
        running_avg_scores.append(running_reward)

        eps = max(eps_end, eps_decay*eps)
        temp = max(temp_end, temp_decay*temp)

        print('\rEpisode {}\tAverage Score: {:.2f}\tRegret: {:.2f}'.format(i_episode, np.mean(scores_window), regrets[-1]), end="")

        if i_episode % 100 == 0:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    return True, scores, regrets, running_avg_scores

### Cartpole Training (Grid Search and Inference)

In [ ]:
cartpole_vars = {
    "BUFFER_SIZE" : [int(100e3), int(250e3), int(500e3)],  # replay buffer size
    "BATCH_SIZE" : [32, 64, 128],                          # minibatch size
    "GAMMA" : [0.99],                                      # discount factor
    "LR" : [1e-3, 1e-4, 1e-5],                             # learning rate
    "UPDATE_EVERY" : [10, 20, 50]                          # how often to update the network (When Q target is present)
}

In [ ]:
env = env_cartpole

# grid_search_results_type1, best_regret = grid_search(env, cartpole_vars, general_vars, "Softmax", type1Update)
# grid_search_results_type2, best_regret = grid_search(env, cartpole_vars, general_vars, "Softmax", type2Update)

threshold = 195
cartpole_vars_optimal = [int(500e3), 64, 0.99, 1e-3, 20, 1, 1, 0]
regrets_type1, scores_type1, running_scores_type1 = inference(env, cartpole_vars_optimal, "Softmax", type1Update)
regrets_type2, scores_type2, running_scores_type2 = inference(env, cartpole_vars_optimal, "Softmax", type2Update)

In [ ]:
plot(regrets_type1, scores_type1, regrets_type2, scores_type2, running_scores_type1, running_scores_type2, threshold)

### Acrobot Training (Grid Search and Inference)

In [ ]:
acrobot_vars = {
    "BUFFER_SIZE" : [int(100e3), int(250e3), int(500e3)],  # replay buffer size
    "BATCH_SIZE" : [32, 64, 128],                          # minibatch size
    "GAMMA" : [0.99],                                      # discount factor
    "LR" : [1e-3, 1e-4, 1e-5],                             # learning rate
    "UPDATE_EVERY" : [10, 20, 50]                          # how often to update the network (When Q target is present)
}

In [ ]:
env = env_acrobot

# grid_search_results_type1, best_regret = grid_search(env, acrobot_vars, general_vars, "Softmax", type1Update)
# grid_search_results_type2, best_regret = grid_search(env, acrobot_vars, general_vars, "Softmax", type2Update)

threshold = -100
acrobot_vars_optimal = [int(500e3), 64, 0.99, 1e-3, 20, 0.01, 0.0001, 0.99]
regrets_type1, scores_type1, running_scores_type1 = inference(env, acrobot_vars_optimal, "Epsilon Greedy", type1Update)
regrets_type2, scores_type2, running_scores_type2 = inference(env, acrobot_vars_optimal, "Epsilon Greedy", type2Update)

In [ ]:
plot(regrets_type1, scores_type1, regrets_type2, scores_type2, running_scores_type1, running_scores_type2, threshold)